In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import transformers
from ipywidgets import interact, interactive
import ipywidgets as widgets

In [2]:
# first, load the tokenizer and bert model
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model = transformers.TFBertForMaskedLM.from_pretrained('bert-base-uncased')

In [3]:
def encode_sentence(sentence):
    # assuming we have things in global namespace, y'know
    # maybe make this in a class
    # NOTE that this olnly works on nominative, for now
    masked_idx = None
    POS = "nom"
    gendered_pronouns = ['he', 'she', 'his', 'hers', 'him', 'her']
    tokenized = tokenizer.tokenize(sentence)
    for i, word in enumerate(tokenized):
        if word in gendered_pronouns:
            #tokenized[i] = "<MASK>"
            tokenized[i] = tokenizer.mask_token
            masked_idx = i
            break
    else:
        print('no gendered pronoun found')
    indexed_text = tokenizer.encode(tokenized, add_special_tokens=True, return_tensors='tf')
    return (indexed_text, masked_idx)
        

In [4]:
def get_gendered_proba(sentence):
    encoded_sent, masked_idx = encode_sentence(sentence)
    predictions = model(encoded_sent)[0]
    she_id = tokenizer.convert_tokens_to_ids('she')
    he_id = tokenizer.convert_tokens_to_ids('he')
    he_pred = predictions[0][masked_idx][he_id]
    she_pred = predictions[0][masked_idx][she_id]
    result = {'he': he_pred.numpy(), 'she': she_pred.numpy()}
    return result

In [5]:
print(get_gendered_proba("The doctor said that he would see the patient soon."))
print(get_gendered_proba("The nurse said that she would see the patient soon."))

{'he': 5.499038, 'she': 4.537035}
{'he': 3.7624776, 'she': 7.6048374}


In [6]:
get_gendered_proba('The computer scientist said that he would not be able to attend the conferenece.')

{'he': 6.5921617, 'she': 4.750877}

In [7]:
print(get_gendered_proba("The nurse treated the patient because she was caring."))
print(get_gendered_proba("The nurse treated the patient because she was screaming."))

{'he': 2.8195734, 'she': 3.1106071}
{'he': 2.1894436, 'she': 2.1881497}


In [8]:
get_gendered_proba("pat said that she was sorry")

{'he': 4.77358, 'she': 4.026822}

# Visualization

In [9]:
attnmodel = transformers.TFBertForMaskedLM.from_pretrained('bert-base-uncased', output_attentions=True)
# note that documentation seems to be wrong -- we want output_attentions, not output_attention

In [10]:
sentence = "The doctor said that he would be late."
encoded_sent, masked_idx = encode_sentence(sentence)
output_attn = attnmodel(encoded_sent)
output = model(encoded_sent)

In [11]:
sentence = "The doctor said that he would be late."
encoded_sent, masked_idx = encode_sentence(sentence)
output_attn = attnmodel(encoded_sent)
words = tokenizer.convert_ids_to_tokens(encoded_sent.numpy()[0])

pd.set_option('precision', 4)

def display_heads(layernum, attnhead = 0):
    for word in encoded_sent[0]:
        words = tokenizer.convert_ids_to_tokens(encoded_sent.numpy()[0])
        df = pd.DataFrame(output_attn[1][layernum][0][attnhead].numpy())
        df.columns = words
        df.index = words
        #return df
        #return df.style.background_gradient(cmap='Blues', high=1.0)
        return (df.style
                .bar(color='lightblue', align='zero', vmax=1.0)
                .set_caption(f'Layer {layernum}, Head {attnhead}')
               )
        

interact(display_heads, 
         layernum=widgets.IntSlider(min=0, max=len(output_attn[1])-1, step=1),
         attnhead=widgets.IntSlider(min=0, max=len(output_attn[1][0][0])-1, step=1)
        )

interactive(children=(IntSlider(value=0, description='layernum', max=11), IntSlider(value=0, description='attn…

<function __main__.display_heads(layernum, attnhead=0)>

In [12]:
def display_heads(layernum, attnhead = 0):
    for word in encoded_sent[0]:
        words = tokenizer.convert_ids_to_tokens(encoded_sent.numpy()[0])
        df = pd.DataFrame(output_attn[1][layernum][0][attnhead].numpy())
        df.columns = words
        df.index = words
        #return df
        return (df.style
                .background_gradient(cmap='Blues', high=1.0)
                .set_caption(f'Layer {layernum}, Head {attnhead}')
               )
        

interact(display_heads, 
         layernum=widgets.IntSlider(min=0, max=len(output_attn[1])-1, step=1),
         attnhead=widgets.IntSlider(min=0, max=len(output_attn[1][0][0])-1, step=1)
        )

interactive(children=(IntSlider(value=0, description='layernum', max=11), IntSlider(value=0, description='attn…

<function __main__.display_heads(layernum, attnhead=0)>